In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape

In [ ]:
base_grande.head()

In [ ]:
ind = []
for i in range (0,4057201) : 
    ind.append(i)
base_grande['ind'] = ind


In [ ]:
base_grande.head()

In [ ]:
liste_diff_patho3 = base_grande['patho_niv3'].unique()
liste_diff_patho2 = base_grande['patho_niv2'].unique()
liste_diff_patho1 = base_grande['patho_niv1'].unique()

In [ ]:
#creation de la colonne cancer pour simplifier la base
"""
set_cancer = set()
for i in patho_3_diff : 
    i.split()
    if 'Cancer' in i or 'cancer' in i :
        set_cancer.add(i)
    else :
        continue
"""
base_grande = base_grande.dropna()

In [ ]:
"""
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    base_grande.loc[:,f"{nom_de_maladie}_ind"] = temporaire
"""
#utiliser get_dummies


base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')

base_grande["classe_age"] = base_grande["cla_age_5"].apply(
    lambda x: np.mean(list(map(lambda y : int(y), x.split("-")))) if "-" in x else (np.nan if x == 'tsage' else 100)
)


In [ ]:
#on refait ce qu'on avait sur le main pour pouvoir avoir les memes variables,
#la meme taille initiale, les memes individus pour pouvoir voir ou ca coince

In [ ]:
#base_grande['Cancer'] = base_grande[liste_cancer_ind].sum(axis = 1)

In [ ]:
base_grande = base_grande.drop(['top', 'cla_age_5', 'patho_niv2', 'patho_niv3', 'annee', 
                                'libelle_classe_age', 'sexe', 'libelle_sexe'], axis = 1)

In [ ]:
liste_var = base_grande.columns

In [ ]:
print(liste_var)

In [ ]:
df_model_cancer = base_grande[['ind', 'dept', 'classe_age', 'indicatrice__Cancers']]

In [ ]:
df_model_cancer.head()

In [ ]:
#https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb

In [ ]:
#debut de la création de carte

In [ ]:
france = gpd.read_file('https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb')

In [ ]:
france.plot()

In [ ]:
df_carto = pd.merge(left = df_model_cancer, right = france, 
                    how = 'left', left_on='dept', right_on='code')

In [ ]:
df_carto['dept'].unique()

In [ ]:
dept_manquants = ['999', '974', '971', '972', '973', '976']

In [ ]:
df_carto = df_carto[~df_carto['dept'].isin(dept_manquants)]

In [ ]:
cancer_dep = df_carto.groupby('dept')['indicatrice__Cancers'].sum().reset_index()

In [ ]:
df_carto = df_carto.merge(cancer_dep, on='dept', suffixes=('', '_sum'))

In [ ]:
df_carto = gpd.GeoDataFrame(df_carto, crs='EPSG:4326')

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 10))
#df_carto.plot(ax=ax, column='Cancers_ind_sum', legend=True, cmap='OrRd')
#plt.show()

In [ ]:
df_carto2 = pd.merge(left = france, right = cancer_dep, 
                    how = 'left', left_on='code', right_on='dept')
fig, ax = plt.subplots(figsize=(10, 10))
df_carto2.plot(ax=ax, column='indicatrice__Cancers', legend=True, cmap='OrRd')
plt.show()

In [ ]:
df_carto2

In [ ]:
base_pop = pd.read_csv("base_pop_dep.csv")

base_pop
df_carto2["pourcentage_cancer"] = (df_carto2["indicatrice__Cancers"] / base_pop["total_habitants"])*100

df_carto2

fig, ax = plt.subplots(figsize=(10, 10))
df_carto2.plot(ax=ax, column='pourcentage_cancer', legend=True, cmap='magma')
plt.show()

#effet de population imp? 
#reste a voir avec regression/acp/clustering
#les departements avec les plus grands taux de cancer semblent etre les departements avec la population agee la plus importante
#on ne remarque pas forcement de suite l effet de la pollution sur le nombre de patients atteints de cancer
#paris aurait ete plus eleve

import folium

centroid = df_carto2.geometry.centroid.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

for i, row in df_carto2.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Dept: {row['nom']}, Pourcentage Cancer: {row['pourcentage_cancer']}",
        tooltip=f"Dept: {row['nom']}, Pourcentage Cancer: {row['pourcentage_cancer']}",
    ).add_to(m)
m
